In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 한글폰트 사용을 위해 설치
# 아래 모듈을 설치하고 불러오면 별도의 한글폰트 설정이 필요 없습니다.
!pip install koreanize-matplotlib

import koreanize_matplotlib

# 시즌 및 팀별로 각 클러스터의 경기 시간 비율을 알려주는 데이터 생성 과정

## 클러스터 결과 각 선수들이 어떤 클러스터 속하는지 나와있는 데이터 불러오기

In [ ]:
df=pd.read_csv('input_df.csv', index_col=0)
df

,PLAYER,SEASON,TEAM,Cluster,MIN
0,Allen Iverson,2000-01,PHI,1,41.9
1,Jerry Stackhouse,2000-01,DET,2,40.1
2,Shaquille O'Neal,2000-01,LAL,1,39.5
3,Kobe Bryant,2000-01,LAL,2,41.0
4,Vince Carter,2000-01,TOR,1,39.7
...,...,...,...,...,...
7549,Mark Pope,2001-02,MIL,0,9.5
7550,Loren Woods,2001-02,MIN,0,8.6
7551,Jud Buechler,2001-02,ORL,0,10.3
7552,Cherokee Parks,2001-02,SAS,0,5.5


In [ ]:
df['SEASON'].unique()

array(['2000-01', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07',
       '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13',
       '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19',
       '2019-20', '2020-21', '2021-22', '2022-23', '2001-02'],
      dtype=object)

## 2000-01 시즌으로 테스트
- 'TEAM'과 'SEASON' 열을 결합하여 새로운 열 'TEAM_SEASON' 생성
- 각 팀별 총 경기 시간 계산
- 팀과 군집별로 경기 시간 비율 합산

In [ ]:
df[df['SEASON'] == '2001-02']

,PLAYER,SEASON,TEAM,Cluster,MIN
7241,Allen Iverson,2001-02,PHI,2,43.7
7242,Shaquille O'Neal,2001-02,LAL,1,36.1
7243,Paul Pierce,2001-02,BOS,1,40.2
7244,Tracy McGrady,2001-02,ORL,2,38.3
7245,Tim Duncan,2001-02,SAS,1,40.6
...,...,...,...,...,...
7549,Mark Pope,2001-02,MIL,0,9.5
7550,Loren Woods,2001-02,MIN,0,8.6
7551,Jud Buechler,2001-02,ORL,0,10.3
7552,Cherokee Parks,2001-02,SAS,0,5.5


In [ ]:
df2000 = df[df['SEASON'] == '2000-01']
df2000

,PLAYER,SEASON,TEAM,Cluster,MIN
0,Allen Iverson,2000-01,PHI,1,41.9
1,Jerry Stackhouse,2000-01,DET,2,40.1
2,Shaquille O'Neal,2000-01,LAL,1,39.5
3,Kobe Bryant,2000-01,LAL,2,41.0
4,Vince Carter,2000-01,TOR,1,39.7
...,...,...,...,...,...
305,Adrian Griffin,2000-01,BOS,0,8.5
306,Greg Foster,2000-01,LAL,0,7.3
307,Mark Madsen,2000-01,LAL,0,9.1
308,Chris Dudley,2000-01,PHX,0,11.5


In [ ]:
# 각 팀별 총 경기 시간 계산
df2000.loc[:, 'min_per'] = df2000.groupby('TEAM')['MIN'].transform(lambda x: x / x.sum())
df2000

<ipython-input-96-5e794226eac0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2000.loc[:, 'min_per'] = df2000.groupby('TEAM')['MIN'].transform(lambda x: x / x.sum())


,PLAYER,SEASON,TEAM,Cluster,MIN,min_per
0,Allen Iverson,2000-01,PHI,1,41.9,0.144234
1,Jerry Stackhouse,2000-01,DET,2,40.1,0.155910
2,Shaquille O'Neal,2000-01,LAL,1,39.5,0.147885
3,Kobe Bryant,2000-01,LAL,2,41.0,0.153501
4,Vince Carter,2000-01,TOR,1,39.7,0.150208
...,...,...,...,...,...,...
305,Adrian Griffin,2000-01,BOS,0,8.5,0.036481
306,Greg Foster,2000-01,LAL,0,7.3,0.027331
307,Mark Madsen,2000-01,LAL,0,9.1,0.034070
308,Chris Dudley,2000-01,PHX,0,11.5,0.042624


In [ ]:
# 팀과 군집별로 경기 시간 비율 합산
team_cluster_df = df2000.pivot_table(index='TEAM', columns='Cluster', values='min_per', aggfunc='sum', fill_value=0)
team_cluster_df

Cluster,0,1,2,3
TEAM,,,,
ATL,0.418893,0.145954,0.095238,0.339915
BOS,0.419313,0.301288,0.179828,0.099571
CHH,0.243441,0.146219,0.302083,0.308256
CHI,0.301571,0.475426,0.000000,0.223002
CLE,0.278354,0.312479,0.000000,0.409167
DAL,0.064924,0.583895,0.177066,0.174115
DEN,0.050518,0.668323,0.070393,0.210766
DET,0.337092,0.113530,0.155910,0.393468
GSW,0.147778,0.427238,0.109788,0.315196


In [ ]:
# 'TEAM'과 'SEASON' 열을 결합하여 새로운 열 'TEAM_SEASON' 생성
df2000.loc[:, 'TEAM_SEASON'] = df2000['SEASON'].str.replace('-', '_') + '_' + df2000['TEAM']

# 각 팀별 총 경기 시간 계산
df2000.loc[:, 'min_per'] = df2000.groupby('TEAM_SEASON')['MIN'].transform(lambda x: x / x.sum())

# 팀과 군집별로 경기 시간 비율 합산 (TEAM_SEASON을 기준으로)
team_cluster_df = df2000.pivot_table(index='TEAM_SEASON', columns='Cluster', values='min_per', aggfunc='sum', fill_value=0)

# 컬럼 이름을 보기 좋게 설정
team_cluster_df.columns = [f'cluster{i}' for i in team_cluster_df.columns]

# 최종 결과 출력
print(team_cluster_df)

             cluster0  cluster1  cluster2  cluster3
TEAM_SEASON                                        
2000_01_ATL  0.418893  0.145954  0.095238  0.339915
2000_01_BOS  0.419313  0.301288  0.179828  0.099571
2000_01_CHH  0.243441  0.146219  0.302083  0.308256
2000_01_CHI  0.301571  0.475426  0.000000  0.223002
2000_01_CLE  0.278354  0.312479  0.000000  0.409167
2000_01_DAL  0.064924  0.583895  0.177066  0.174115
2000_01_DEN  0.050518  0.668323  0.070393  0.210766
2000_01_DET  0.337092  0.113530  0.155910  0.393468
2000_01_GSW  0.147778  0.427238  0.109788  0.315196
2000_01_HOU  0.238386  0.274246  0.162592  0.324776
2000_01_IND  0.206700  0.638751  0.000000  0.154549
2000_01_LAC  0.209758  0.110509  0.300667  0.379066
2000_01_LAL  0.477724  0.252714  0.153501  0.116061
2000_01_MIA  0.208548  0.791452  0.000000  0.000000
2000_01_MIL  0.162531  0.413978  0.157982  0.265509
2000_01_MIN  0.314931  0.104854  0.280474  0.299741
2000_01_NJN  0.300000  0.315000  0.275385  0.109615
2000_01_NYK 

<ipython-input-7-5523eb2e4883>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2000.loc[:, 'min_per'] = df2000.groupby('TEAM_SEASON')['MIN'].transform(lambda x: x / x.sum())


## 모든 시즌에 대해 진행

In [ ]:
result_list=[]
# 2000년부터 2022년까지 반복
for year in range(2000, 2023):
    # 해당 연도 데이터만 필터링하여 데이터프레임 생성
    season_str = f'{year}-{str(year+1)[-2:]}'  # 시즌 문자열 예: '2000-01'
    df_season = df[df['SEASON'] == season_str].copy()  # 해당 시즌의 데이터프레임 생성

    # 'TEAM_SEASON' 열을 만들어서 팀 이름과 시즌을 합침
    df_season.loc[:, 'TEAM_SEASON'] = df_season['SEASON'].str.replace('-', '_') + '_' + df_season['TEAM']

    # 각 팀별로 총 경기 시간에 대한 비율을 계산
    df_season.loc[:, 'min_per'] = df_season.groupby('TEAM_SEASON')['MIN'].transform(lambda x: x / x.sum())

    # 팀과 군집별로 경기 시간 비율 합산 (TEAM_SEASON을 기준으로)
    team_cluster_df = df_season.pivot_table(index='TEAM_SEASON', columns='Cluster', values='min_per', aggfunc='sum', fill_value=0)

    # 컬럼 이름을 보기 좋게 설정
    team_cluster_df.columns = [f'cluster{i}' for i in team_cluster_df.columns]

    # 각 시즌의 결과를 리스트에 추가
    result_list.append(team_cluster_df)

# 결과 리스트를 모두 합친 후 최종 데이터프레임으로 변환 (여러 시즌 데이터를 하나로)
final_result_df = pd.concat(result_list)

# 최종 결과 출력
print(final_result_df)

             cluster0  cluster1  cluster2  cluster3
TEAM_SEASON                                        
2000_01_ATL  0.418893  0.145954  0.095238  0.339915
2000_01_BOS  0.419313  0.301288  0.179828  0.099571
2000_01_CHH  0.243441  0.146219  0.302083  0.308256
2000_01_CHI  0.301571  0.475426  0.000000  0.223002
2000_01_CLE  0.278354  0.312479  0.000000  0.409167
...               ...       ...       ...       ...
2022_23_SAC  0.268489  0.592444  0.139068  0.000000
2022_23_SAS  0.446910  0.426023  0.127067  0.000000
2022_23_TOR  0.360516  0.296599  0.342884  0.000000
2022_23_UTA  0.315236  0.587963  0.000000  0.096801
2022_23_WAS  0.373395  0.333452  0.124822  0.168331

[686 rows x 4 columns]


In [ ]:
final_result_df

,cluster0,cluster1,cluster2,cluster3
TEAM_SEASON,,,,
2000_01_ATL,0.418893,0.145954,0.095238,0.339915
2000_01_BOS,0.419313,0.301288,0.179828,0.099571
2000_01_CHH,0.243441,0.146219,0.302083,0.308256
2000_01_CHI,0.301571,0.475426,0.000000,0.223002
2000_01_CLE,0.278354,0.312479,0.000000,0.409167
...,...,...,...,...
2022_23_SAC,0.268489,0.592444,0.139068,0.000000
2022_23_SAS,0.446910,0.426023,0.127067,0.000000
2022_23_TOR,0.360516,0.296599,0.342884,0.000000


In [ ]:
final_result_df.to_csv('final_result_df.csv')

# playoff 진출 여부 변수 추가 과정

## season별 boxscore 불러와서 playoff 진출팀 확인 - 2000-01 시즌으로 테스트

In [ ]:
df_2000_01 = pd.read_csv('team_boxscore_2000-01.csv')
df_2000_01

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Season
0,PHI,PHI vs. LAL,06/15/2001,L,240,96,37,90,41.1,4,...,20,31,51,23,6,4,12,32,-12,2000-01
1,LAL,LAL @ PHI,06/15/2001,W,240,108,32,71,45.1,12,...,14,33,47,21,6,10,14,22,12,2000-01
2,PHI,PHI vs. LAL,06/13/2001,L,240,86,33,77,42.9,1,...,8,29,37,15,9,1,9,27,-14,2000-01
3,LAL,LAL @ PHI,06/13/2001,W,240,100,36,72,50.0,10,...,12,31,43,24,6,6,14,22,14,2000-01
4,PHI,PHI vs. LAL,06/10/2001,L,240,91,33,80,41.3,2,...,13,29,42,17,5,6,16,22,-5,2000-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,UTA,UTA vs. DAL,04/21/2001,W,240,88,33,80,41.3,3,...,19,29,48,24,7,5,15,22,2,2000-01
138,SAS,SAS vs. MIN,04/21/2001,W,240,87,33,66,50.0,4,...,8,34,42,18,3,6,20,20,5,2000-01
139,IND,IND @ PHI,04/21/2001,W,240,79,31,87,35.6,5,...,11,32,43,20,4,5,7,18,1,2000-01
140,CHH,CHH @ MIA,04/21/2001,W,240,106,37,77,48.1,7,...,9,30,39,21,17,4,15,26,26,2000-01


In [ ]:
final_result_df = pd.read_csv('final_result_df.csv')

In [ ]:
playdic={}
playdic['2000_01']=df_2000_01['Team'].unique()
playdic

{'2000_01': array(['PHI', 'LAL', 'MIL', 'SAS', 'CHH', 'TOR', 'DAL', 'SAC', 'NYK',
        'UTA', 'IND', 'PHX', 'ORL', 'MIN', 'POR', 'MIA'], dtype=object)}

In [ ]:
playoff_dic = {'2000_01':df_2000_01['Team'].unique()}
playoff_dic

{'2000_01': array(['PHI', 'LAL', 'MIL', 'SAS', 'CHH', 'TOR', 'DAL', 'SAC', 'NYK',
        'UTA', 'IND', 'PHX', 'ORL', 'MIN', 'POR', 'MIA'], dtype=object)}

In [ ]:
# 2000_01 시즌의 플레이오프 팀 목록 가져오기
playoff_teams_2000_01 = playoff_dic['2000_01']

# 새로운 열 playoffs를 추가하기 위해 조건 설정
# 'TEAM_SEASON' 열에서 '2000_01' 문자열을 포함하는 데이터프레임만 추출
filtered_df = final_result_df[final_result_df['TEAM_SEASON'].str.contains('2000_01')]

# 추출된 데이터프레임에서 index.map을 사용하여 playoffs 열 추가
filtered_df['playoffs'] = filtered_df['TEAM_SEASON'].map(lambda x: 1 if x[-3:] in playoff_teams_2000_01 else 0)

# 결과 확인
print(filtered_df)

    TEAM_SEASON  cluster0  cluster1  cluster2  cluster3  playoffs
0   2000_01_ATL  0.418893  0.145954  0.095238  0.339915         0
1   2000_01_BOS  0.419313  0.301288  0.179828  0.099571         0
2   2000_01_CHH  0.243441  0.146219  0.302083  0.308256         1
3   2000_01_CHI  0.301571  0.475426  0.000000  0.223002         0
4   2000_01_CLE  0.278354  0.312479  0.000000  0.409167         0
5   2000_01_DAL  0.064924  0.583895  0.177066  0.174115         1
6   2000_01_DEN  0.050518  0.668323  0.070393  0.210766         0
7   2000_01_DET  0.337092  0.113530  0.155910  0.393468         0
8   2000_01_GSW  0.147778  0.427238  0.109788  0.315196         0
9   2000_01_HOU  0.238386  0.274246  0.162592  0.324776         0
10  2000_01_IND  0.206700  0.638751  0.000000  0.154549         1
11  2000_01_LAC  0.209758  0.110509  0.300667  0.379066         0
12  2000_01_LAL  0.477724  0.252714  0.153501  0.116061         1
13  2000_01_MIA  0.208548  0.791452  0.000000  0.000000         1
14  2000_0

<ipython-input-29-716dfc88b63d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['playoffs'] = filtered_df['TEAM_SEASON'].map(lambda x: 1 if x[-3:] in playoff_teams_2000_01 else 0)


In [ ]:
filtered_df['playoffs'].value_counts()

,count
playoffs,
1,16
0,14


## 모든 시즌에 대해서 진행

In [ ]:
seasons = [f"{year}_{str(year + 1)[-2:]}" for year in range(2000, 2023)]

playoff_dic = {}
columns = final_result_df.columns.tolist()
columns.append('playoffs')
final_result_df_with_playoff = pd.DataFrame(columns=columns)

In [ ]:
final_result_df_with_playoff

,TEAM_SEASON,cluster0,cluster1,cluster2,cluster3,playoffs


In [ ]:
for season in seasons:
    season_bar = season.replace('_', '-')
    boxscore = pd.read_csv(f'team_boxscore_{season_bar}.csv')
    #playoff_dic[season]=boxscore['Team'].unique()

    # 'TEAM_SEASON' 열에서 ex)'2000_01' 문자열을 포함하는 데이터프레임만 추출
    filtered_df = final_result_df[final_result_df['TEAM_SEASON'].str.contains(season)]

    # 해당 시즌의 플레이오프 팀 목록 가져오기
    #playoff_teams = playoff_dic[season]
    playoff_teams = boxscore['Team'].unique()
    print(f"The number of teams for the season {season}: ", len(playoff_teams))

    # 추출된 데이터프레임에서 ['TEAM_SEASON'].map을 사용하여 playoffs 열 추가
    filtered_df.loc[:, 'playoffs'] = filtered_df['TEAM_SEASON'].map(lambda x: 1 if x[-3:] in playoff_teams else 0)

    # 데이터프레임 합치기
    final_result_df_with_playoff = pd.concat([final_result_df_with_playoff, filtered_df])

The number of teams for the season 2000_01:  16
The number of teams for the season 2001_02:  16
The number of teams for the season 2002_03:  16
The number of teams for the season 2003_04:  16
The number of teams for the season 2004_05:  16
The number of teams for the season 2005_06:  16
The number of teams for the season 2006_07:  16
The number of teams for the season 2007_08:  16
The number of teams for the season 2008_09:  16
The number of teams for the season 2009_10:  16
The number of teams for the season 2010_11:  16
The number of teams for the season 2011_12:  16
The number of teams for the season 2012_13:  16
The number of teams for the season 2013_14:  16
The number of teams for the season 2014_15:  16
The number of teams for the season 2015_16:  16
The number of teams for the season 2016_17:  16
The number of teams for the season 2017_18:  16
The number of teams for the season 2018_19:  16
The number of teams for the season 2019_20:  16
The number of teams for the season 2020_

<ipython-input-45-8bde6d741268>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'playoffs'] = filtered_df['TEAM_SEASON'].map(lambda x: 1 if x[-3:] in playoff_teams else 0)
<ipython-input-45-8bde6d741268>:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_result_df_with_playoff = pd.concat([final_result_df_with_playoff, filtered_df])
<ipython-input-45-8bde6d741268>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [ ]:
final_result_df_with_playoff

,TEAM_SEASON,cluster0,cluster1,cluster2,cluster3,playoffs
0,2000_01_ATL,0.418893,0.145954,0.095238,0.339915,0
1,2000_01_BOS,0.419313,0.301288,0.179828,0.099571,0
2,2000_01_CHH,0.243441,0.146219,0.302083,0.308256,1
3,2000_01_CHI,0.301571,0.475426,0.000000,0.223002,0
4,2000_01_CLE,0.278354,0.312479,0.000000,0.409167,0
...,...,...,...,...,...,...
681,2022_23_SAC,0.268489,0.592444,0.139068,0.000000,1
682,2022_23_SAS,0.446910,0.426023,0.127067,0.000000,0
683,2022_23_TOR,0.360516,0.296599,0.342884,0.000000,0
684,2022_23_UTA,0.315236,0.587963,0.000000,0.096801,0


In [ ]:
'''# 'TEAM_SEASON' 열을 가장 먼저 오는 열로 설정
columns = ['TEAM_SEASON'] + [col for col in final_result_df_with_playoff.columns if col != 'TEAM_SEASON']
final_result_df_with_playoff = final_result_df_with_playoff[columns]

# 최종 결과 출력
print(final_result_df_with_playoff)'''

             TEAM_SEASON  cluster0  cluster1  cluster2  cluster3 playoffs
2000_01_ATL          NaN  0.418893  0.145954  0.095238  0.339915        0
2000_01_BOS          NaN  0.419313  0.301288  0.179828  0.099571        0
2000_01_CHH          NaN  0.243441  0.146219  0.302083  0.308256        1
2000_01_CHI          NaN  0.301571  0.475426  0.000000  0.223002        0
2000_01_CLE          NaN  0.278354  0.312479  0.000000  0.409167        0
...                  ...       ...       ...       ...       ...      ...
681          2022_23_SAC  0.268489  0.592444  0.139068  0.000000        1
682          2022_23_SAS  0.446910  0.426023  0.127067  0.000000        0
683          2022_23_TOR  0.360516  0.296599  0.342884  0.000000        0
684          2022_23_UTA  0.315236  0.587963  0.000000  0.096801        0
685          2022_23_WAS  0.373395  0.333452  0.124822  0.168331        0

[3430 rows x 6 columns]


In [ ]:
final_result_df_with_playoff.to_csv('final_result_df_with_playoff.csv')

In [ ]:
# 'TEAM_SEASON' 열에서 앞 7자를 추출하여 'SEASON' 열에 할당
final_result_df_with_playoff.loc[:, 'SEASON'] = final_result_df_with_playoff['TEAM_SEASON'].str[:7]
final_result_df_with_playoff

,TEAM_SEASON,cluster0,cluster1,cluster2,cluster3,playoffs,SEASON
0,2000_01_ATL,0.418893,0.145954,0.095238,0.339915,0,2000_01
1,2000_01_BOS,0.419313,0.301288,0.179828,0.099571,0,2000_01
2,2000_01_CHH,0.243441,0.146219,0.302083,0.308256,1,2000_01
3,2000_01_CHI,0.301571,0.475426,0.000000,0.223002,0,2000_01
4,2000_01_CLE,0.278354,0.312479,0.000000,0.409167,0,2000_01
...,...,...,...,...,...,...,...
681,2022_23_SAC,0.268489,0.592444,0.139068,0.000000,1,2022_23
682,2022_23_SAS,0.446910,0.426023,0.127067,0.000000,0,2022_23
683,2022_23_TOR,0.360516,0.296599,0.342884,0.000000,0,2022_23
684,2022_23_UTA,0.315236,0.587963,0.000000,0.096801,0,2022_23


In [ ]:
final_result_df_with_playoff['SEASON'].value_counts()

,count
SEASON,
2011_12,30
2012_13,30
2021_22,30
2020_21,30
2019_20,30
2018_19,30
2017_18,30
2016_17,30
2015_16,30


In [ ]:
final_result_df_with_playoff[final_result_df_with_playoff['TEAM_SEASON']=='2000_01_ATL']

,TEAM_SEASON,cluster0,cluster1,cluster2,cluster3,playoffs,SEASON
0,2000_01_ATL,0.0179,0.006237,0.00407,0.014525,0,2000_01
0,2000_01_ATL,0.0179,0.006237,0.00407,0.014525,0,2000_01
0,2000_01_ATL,0.0179,0.006237,0.00407,0.014525,0,2000_01
